In [16]:
# T:
# T2:
# P1: This is the x-coordinate of the parent position
# P2: This is the x-coordinate of the position
# Branch:
# Stat/Dyn - stat shows the whole org chart, dyn only allows nodes to expand/contract

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# https://stackoverflow.com/questions/44549110/python-loop-through-excel-sheets-place-into-one-df

path = 'C:/Users/jlahrman/OneDrive - LMI/Documents/DoS_Viz/Data/2021_04_08/'

sheets = pd.read_excel(path + 'dept_row_data_for_testing.xlsx', sheet_name = None)

# blank data frame to append every sheet
final_data = pd.DataFrame()

# rename level to 'end_node_level'
for name, sheet in sheets.items():
    sheet['sheet'] = name
    sheet = sheet.rename(columns=lambda x: x.split('\n')[-1])
    sheet.columns = ['row_id','end_node_level','position_category','position_name','details', 'detail_count', 'pos_no','inner_count','reports_to_row_id','to_plot', 'department']
    final_data = final_data.append(sheet)
    
#final_data['details'] = re.sub(\"[^\\w\\d'\\s]+\", '')

# start a loop here to handle each individual department, and a data frame to hold all final data
final_curve_data = pd.DataFrame()
department_list = final_data.department.unique()

for x in range(len(department_list)):
    
    department_data = final_data[final_data.department == department_list[x]]
    
    print(department_list[x] + ":" + str(len(department_data)))
    
    # iterate through the levels in reverse order - will probably be better to do it this way when trying to center the parent nodes

    # determine maximum level in the data
    max_level = int(department_data["end_node_level"].max())
    # create blank data frame to append the records with updated fields
    final_data_with_coords = pd.DataFrame()
    # create a counter for the end_node_id, this just needs to stay unique in the data frame
    end_node_id = 1

    # Set a value to iterate the P2 coordinate for each row
    P2_static_iterator = 3
    # Set the value of T to interate both T and T2
    T_initializer = 6
    T_iterator = 6 # I think this should be right, it's just 6?
    
    # set up a list to eventually turn into a data frame that we'll use to center the points on each row
    level_max = []

    for i in range(max_level):
        level_i = max_level - i
        # we also only need to attach these coordinates to values that we want to plot
        subset = department_data[(department_data.end_node_level == level_i) & (department_data.to_plot == 1)]
        # start P2 at 0, then append values, iterating by the P2_iterator value
        P2_static_coord = 0
        P2_static_set = []
        end_node_id_set = []
        # for each entry, increment P2 (the x-coordinate of the node) by 3
        # for each entry, increment end_node_id by 1
        for j in range(len(subset)):
            P2_static_set.append(P2_static_coord)
            end_node_id_set.append(end_node_id)
            P2_static_coord += P2_static_iterator
            end_node_id += 1
        if len(subset) > 0:
            # have to subtract that iterator back out because it already added the iterator value before the loop ended...
            level_max.append([department_list[x],level_i,P2_static_coord - P2_static_iterator]) 

    # Append the individual lists we just created as new columns of the data set using 'assign'
        subset = subset.assign(P2_static = P2_static_set)
        subset = subset.assign(end_node_id = end_node_id_set)
        # add the T_initializer value for the initial T
        subset = subset.assign(T = -T_initializer)
        final_data_with_coords = final_data_with_coords.append(subset)

    # here is where I use that level_max data that we compiled above to adjust the P2s to be centered on the page 
    # we have to finalize the P2s here because everyone's P1 coordinate is based on the P2 of their parent position
    # determine the maximum P2 value for each level
    level_max = pd.DataFrame(level_max,columns=['department','level','max_P2_static'])
    # now find the overall maximum value for the department
    overall_max = level_max['max_P2_static'].max()
    # to horizontally center, determine a level-specific adjustment factor
    level_max['P2_static_adjustment'] = (overall_max - level_max['max_P2_static'])/2

    # maybe this could be combined with the loop below but I'll keep separate for now.
    # go through each record in final_data_with_coords, look up the appropriate P2_adjustment for the level, and add it to P2

    for j in range(len(final_data_with_coords)):
        # go through each row individually, save the row as a series
        lookup = final_data_with_coords.iloc[j]
        # find the P2_static value for that value
        end_node_level_lookup = lookup.get(key = 'end_node_level')
        # now find it in the level_max_data frame, and add that value to the existing value of P2
        final_data_with_coords.at[final_data_with_coords.index[j], 'P2_static'] += level_max[level_max['level']==end_node_level_lookup]['P2_static_adjustment']
    
    # now we need to get some information from the position to which each position reports to:
    # to get the start node level and id
    # reports to details
    # And P2 of the parent's node becomes P1 of the position
    # we have to look up the end node level and id of the position listed in the 'reports_to_row_id' field

    reports_to_details_set = []
    P1_static_set = []
    sni_set = []
    snl_set = []
    T2_set = []

    for j in range(len(final_data_with_coords)):
        # go through each row individually, save the row as a series
        lookup = final_data_with_coords.iloc[j]
        # now determine the reports_to_row_id value within the series
        lookup_row_id = lookup.get(key = 'reports_to_row_id')

        if pd.isna(lookup_row_id):
            # if this position doesn't have a known report up, just list the position's name as the one it reports to
            # Start node level and id are the same as the position's end node level and id - hold on there, it needs to be decremented back one but we'll handle that later
            # And we can't create T2 until here, the first value has to be based on its parent's position unless we want to increment backwards
            rtd_item = lookup.get(key = 'details')
            P1_static_item = lookup.get(key = 'P2_static')
            sni_item = lookup.get(key = 'end_node_id')
            snl_item = lookup.get(key = 'end_node_level')
            T2_item = -T_initializer + lookup.get(key = 'end_node_level') * 2 * T_iterator
            #    subset = subset.assign(T2 = -T_initializer + (level_i-1)*(T_iterator)*2)
        else:
            # if this position does have a report up, narrow the original data frame to the row_id with the lookup value
            reports_to_info = final_data_with_coords[final_data_with_coords['row_id'] == lookup_row_id]
            rtd_item = reports_to_info['details'].iloc[0]
            P1_static_item = reports_to_info['P2_static'].iloc[0]
            sni_item = reports_to_info['end_node_id'].iloc[0]
            snl_item = reports_to_info['end_node_level'].iloc[0]
            T2_item = -T_initializer + reports_to_info['end_node_level'].iloc[0] * 2 * T_iterator
        reports_to_details_set.append(rtd_item)
        P1_static_set.append(P1_static_item)
        sni_set.append(sni_item)
        snl_set.append(snl_item)
        T2_set.append(T2_item)

    final_data_with_coords = final_data_with_coords.assign(reports_to_details = reports_to_details_set)
    final_data_with_coords = final_data_with_coords.assign(P1_static = P1_static_set)
    final_data_with_coords = final_data_with_coords.assign(start_node_id = sni_set)
    final_data_with_coords = final_data_with_coords.assign(start_node_level = snl_set)
    final_data_with_coords = final_data_with_coords.assign(T2 = T2_set)
    
    # Build a loop that creates P1_dynamic and P2_dynamic, as well as compiles the Path.
    # Also add an empty column of lists to which end_node_ids can be appended to build the path variable
    
    final_data_with_coords['path_builder'] = [[] for x in range(len(final_data_with_coords))]
    
    # start by setting P1_ and P2_dynamic to equal their static counterparts for level 1 positions, and add the end_node_id to the path
    # actually, let's do that for any node whose end_node_id = start_node id (ie, does not report to anyone)
    for y in range(len(final_data_with_coords)):
#        if final_data_with_coords.at[final_data_with_coords.index[y], 'end_node_level'] == 1:
        if final_data_with_coords.at[final_data_with_coords.index[y], 'end_node_level'] == final_data_with_coords.at[final_data_with_coords.index[y], 'start_node_level']:
            final_data_with_coords.at[final_data_with_coords.index[y], 'P1_dynamic'] = final_data_with_coords.at[final_data_with_coords.index[y], 'P1_static']
            final_data_with_coords.at[final_data_with_coords.index[y], 'P2_dynamic'] = final_data_with_coords.at[final_data_with_coords.index[y], 'P2_static']
            final_data_with_coords.at[final_data_with_coords.index[y], 'path_builder'].append(int(final_data_with_coords.at[final_data_with_coords.index[y], 'end_node_id']))
    
    # for levels 2 - n, find the number of records that have that level and have a report (IE, they do NOT report to themselves)
    P2_dataset = final_data_with_coords[(final_data_with_coords['end_node_level'] > 1) & (final_data_with_coords['end_node_id'] != final_data_with_coords['start_node_id'])]
    P2_dataset_levels = P2_dataset.end_node_level.unique()
    P2_dataset_levels.sort()
    
    # Joel: Start here, you're ready to go through the P2 datasets individually.
    # Filter final_data_with_coords by the individual P2_dataset_levels values and end_node_id <> start_node_id and also determine the length
    
    for x in range(len(P2_dataset_levels)):
        iterator = 0
        for y in range(len(final_data_with_coords)):
            if (final_data_with_coords.at[final_data_with_coords.index[y], 'end_node_level'] == P2_dataset_levels[x]) & (final_data_with_coords.at[final_data_with_coords.index[y], 'end_node_level'] != final_data_with_coords.at[final_data_with_coords.index[y], 'start_node_level']):
                # look up the P2_dynamic of the parent
                lookup = final_data_with_coords.iloc[y]
                lookup_row_id = lookup.get(key = 'reports_to_row_id')
                reports_to_info = final_data_with_coords[final_data_with_coords['row_id'] == lookup_row_id]
                # assign it to both the P1_dynamic and P2_dynamic of the child
                # got to restart the iterator if the parent id changes, need to somehow identify that set.
                # maybe easier to get this done, then go back and subset every combination and somehow iterate through?
                final_data_with_coords['P1_dynamic'].iloc[y] = 1234
                final_data_with_coords['P2_dynamic'].iloc[y] = reports_to_info['P2_dynamic'].iloc[0]
                
                # Bring in the Path from the parent
                final_data_with_coords['path_builder'].iloc[y] = reports_to_info['path_builder'].iloc[0]
                # append the end_node_id of the child - why is it bringing in everything?
                final_data_with_coords['path_builder'].iloc[y].append(int(final_data_with_coords['end_node_id'].iloc[y]))
#                final_data_with_coords.at[final_data_with_coords.index[y], 'path_builder'].append(int(final_data_with_coords.at[final_data_with_coords.index[y], 'end_node_id']))
#                final_data_with_coords['path_builder'].iloc[y].append(final_data_with_coords['end_row_id'].iloc[y])
                iterator += 1

# Then increment P2_dynamic by 1. Should be easy to double check. Adjust it by subtracting the length of the data - 1, then dividing by two

    # If a position does not report to another position, then T2 needs to be incremented down so that the Y value of the start node gets incremented down to the previous level
    # final_data_with_coords['T2'].where(~(final_data_with_coords.end_node_level == final_data_with_coords.start_node_level), other=final_data_with_coords.T2 + 2 * T_initializer, inplace=True)
    final_data_with_coords.to_csv('final_data_with_coords' + department_list[x] + '.csv',index=False)

    branch_set = []

    for j in range(len(final_data_with_coords)):
        lookup = final_data_with_coords.iloc[j]
        branch_item = "Level " + str(int(lookup.get(key = 'start_node_level'))) + " (" + str(int(lookup.get(key = 'start_node_id'))) + ") - Level " + str(int(lookup.get(key = 'end_node_level'))) + " (" + str(int(lookup.get(key = 'end_node_id'))) + ")"
        branch_set.append(branch_item)

    final_data_with_coords = final_data_with_coords.assign(branch = branch_set)

    # add in a T2 iterator
    final_data_with_coords['T2_iterator'] = .25 * (final_data_with_coords['end_node_level'] - final_data_with_coords['start_node_level'])

    department_curve_data = pd.DataFrame()
    number_of_rows = 49
#    final_data_with_coords.to_csv('final_data_with_coords' + department_list[x] + '.csv',index=False)

    # annoying but I'm going to create a unique list of IDs and then filter the data frame by every unique value
    # should have the same effect as selecting individual rows (which is a pain because those rows are saved as series and I lose the column names)

    # create 49 of each record:
    for ind in final_data_with_coords.index:
        curve_data = final_data_with_coords.loc[final_data_with_coords.index.repeat(number_of_rows)]

    # now go through each individual set of 49 records and iterate T1 (by 0.25) and T2 (by 0.25 * the number of levels)

    row_id_list = final_data_with_coords['row_id'].tolist()
    for k in range(len(row_id_list)):
        iterated_data = curve_data[curve_data['row_id'] == row_id_list[k]]
        T_new_set = []
        T2_new_set = []
        for l in range(len(iterated_data)):
            lookup = iterated_data.iloc[l]
            T_new_item = lookup.get(key = 'T') + l * .25
            T2_new_item = lookup.get(key = 'T2') + l * lookup.get(key = 'T2_iterator')
            T_new_set.append(T_new_item)
            T2_new_set.append(T2_new_item)
        iterated_data = iterated_data.assign(T = T_new_set)
        iterated_data = iterated_data.assign(T2 = T2_new_set)
        department_curve_data = department_curve_data.append(iterated_data)
    
    # now append back the records that are not being appended, so that we can still include them in headounts
    unplotted_data = department_data[department_data.to_plot != 1]
    department_curve_data = department_curve_data.append(unplotted_data)
    final_curve_data = final_curve_data.append(department_curve_data)

# remove unnecessary fields
#final_curve_data.drop(columns="T2_iterator", inplace = True)

# attempt to get rid of trailing zeroes (.0)
#final_curve_data_fixed = final_curve_data.applymap(lambda cell: int(cell) if str(cell).endswith('.0') else cell)

# publish final curve data
final_curve_data.to_csv('final_curve_data.csv',index=False)
#final_curve_data_fixed.to_csv('final_curve_data_fixed.csv',index=False)

DoD:882
DoC:68
DoA:309
ED simple:295
DHS full:310


IndexError: index 34 is out of bounds for axis 0 with size 5

In [9]:
# come up with a way to set the P2 increment to make the plot a roughly consistently rectangular shape
# space when reporting to different positions
# don't create 49 records for positions that don't report up, just one
# create Path logic
# Tableau - exploding nodes?

In [534]:
#final_data_with_coords.to_csv('final_data_with_coords.csv',index=False)

#final_data.to_csv('2_DoD_2_level_example_final_data.csv',index=False)

#iterated_data.to_csv('iterated_data.csv',index=False)